In [38]:
PATH = '..'
DATA_SOURCE = f'{PATH}/data/source'
DATA_PROCESSED = f'{PATH}/data/processed'

In [39]:
import os
import pandas as pd
import numpy as np
import polars as pl
from IPython.display import display, Markdown
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import json
from tqdm.notebook import tqdm

import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util
import hdbscan
from umap import UMAP
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import pymorphy2
from razdel import sentenize
import eli5
from sklearn.metrics.pairwise import cosine_similarity

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=16)
tqdm.pandas()

from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline, make_union
import torch
from scipy.sparse import coo_matrix

from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [40]:
def plot_wordcloud(list_words):
    wordcloud = WordCloud(
        width = 2000, 
        height = 1500, 
        random_state=1, 
        background_color='black', 
        margin=20, 
        colormap='Pastel1', 
        collocations=False
    )
    frequencies = pd.Series(list_words).value_counts()
    plt.figure(figsize=(40, 30))
    plt.imshow(wordcloud.generate_from_frequencies(frequencies))
    plt.axis("off");

In [41]:
class Tokenizer:
    def __init__(self, pattern='[^a-zа-яё]+', norm=False, stop_words=[]):
        # pattern='[^a-zа-яё]+'
        # pattern='\W'
        self._pattern = re.compile(pattern)
        self._norm = norm
        self._morph = pymorphy2.MorphAnalyzer() if norm else None
        self._stop_words = stop_words

    def __call__(self, text):
        tokens = [token for token in self._pattern.split(text.lower()) if token != '']
        if self._norm:
            tokens = [self._morph.parse(token)[0].normal_form for token in tokens]
            
        tokens = [token for token in tokens if len(token) > 2 and token not in self._stop_words]
        return tokens

def sentenizer(text):
    text = re.sub(r"([a-zа-я\)]\s+)([А-Я])", r"\1. \2", text)
    text = re.sub(r"\s+\.", r".", text)
    text_sent = sentenize(text)
    text_sent = [sent.text for sent in text_sent]
    return text_sent

# Чтение данных

In [42]:
df_vacancy = pd.read_pickle(f'{DATA_PROCESSED}/df_vacancy.pickle')
df_resume = pd.read_pickle(f'{DATA_PROCESSED}/df_resume.pickle')
df_pairs = pd.read_pickle(f'{DATA_PROCESSED}/df_pairs.pickle')

# Разбивка вакансии и резюме на кластеры

In [43]:
titles = [
    ['ИТ-Лидер команды', 'ИТ-лидер', 'Team Lead', 'Руководитель группы разработки'],
    ['Senior Java-разработчик', 'Java разработчик senior', 'Java-разработчик проект АБС', 'Senior Java-разработчик проект брокерское обслуживание', 'Java developer', 'Java разработчик команда Инвестиции', 'Android разработчик', 'Java-разработчик', 'Java-разработчик', 'Java разработчик'],
    ['Системный аналитик', 'Системный аналитик', 'Системный аналитик финтех', 'Системный аналитик комманда Залоги'],
    ['DevOps инженер', 'DevOps'],
    ['Аналитик DWH "Hadoop"', 'Аналитик DWH', 'Ведущий/ Главный аналитик DWH', 'Архитектор/Системный аналитик DWH ДИР'],
    ['Ведущий разработчик ETL "Hadoop"', 'ETL Разработчик (ДИР)', ],
    ['Product manager', 'Product Manager'],
    ['Тестировщик'],
    ['Frontend Developer'],
    ['Python Developer'],
]
df_titles = pd.Series(titles).to_frame('title_vacancy')
df_titles['id_cluster'] = range(len(df_titles))
df_titles = df_titles.explode('title_vacancy')

clms = ['title_vacancy', 'title_resume', 'label', 'id_cluster', 'split']
df_titles = df_pairs \
    .merge(df_vacancy[['uuid', 'title']], left_on='uuid_vacancy', right_on='uuid', how='left') \
    .merge(df_resume[['uuid', 'title']], left_on='uuid_resume', right_on='uuid', suffixes=['_vacancy', '_resume'], how='left') \
    .merge(df_titles, on='title_vacancy', how='left')[clms]

/tmp/ipykernel_1264180/2188640619.py:18: FutureWarning: Passing 'suffixes' which cause duplicate columns {'uuid_vacancy'} in the result is deprecated and will raise a MergeError in a future version.
  df_titles = df_pairs \


In [44]:
df_titles.isna().sum()

title_vacancy      0
title_resume       0
label            113
id_cluster         0
split              0
dtype: int64

In [45]:
df_titles.query('title_resume == ""')

,title_vacancy,title_resume,label,id_cluster,split


In [46]:
cluster_names = [
    'Team Lead',
    'Java-разработчик',
    'Системный аналитик',
    'DevOps',
    'Аналитик DWH',
    'ETL Разработчик',
    'Product Manager',
    'Тестировщик',
    'Frontend Developer',
    'Python Developer',
]
cluster2title = pd.Series(dict(enumerate(cluster_names)))

title2cluster = pd.concat([
    df_titles.set_index('title_vacancy')['id_cluster'],
    df_titles.set_index('title_resume')['id_cluster']
])
title2cluster = pd.Series(title2cluster.to_dict())

In [47]:
print(*cluster_names, sep='\n')

Team Lead
Java-разработчик
Системный аналитик
DevOps
Аналитик DWH
ETL Разработчик
Product Manager
Тестировщик
Frontend Developer
Python Developer


In [48]:
for id_cluster, subset in df_titles.query('label == 1').groupby('id_cluster'):
    display(subset)
    print()

,title_vacancy,title_resume,label,id_cluster,split
162,Руководитель группы разработки,Team Lead Java,1.0,0,train
215,Руководитель группы разработки,Team Lead,1.0,0,train
236,ИТ-Лидер команды,Team Lead,1.0,0,train
287,ИТ-лидер,Team Lead,1.0,0,train
296,ИТ-Лидер команды,Ведущий Java-разработчик,1.0,0,train
469,Руководитель группы разработки,Team Lead,1.0,0,train
493,ИТ-лидер,Team Lead,1.0,0,train
589,Руководитель группы разработки,Lead,1.0,0,train
645,ИТ-лидер,Tech lead,1.0,0,train
652,ИТ-Лидер команды,Lead,1.0,0,train


,title_vacancy,title_resume,label,id_cluster,split
2,Java-разработчик проект АБС,Главный специалист поддержки банковских систем,1.0,1,train
20,Java разработчик senior,Java developer,1.0,1,train
22,Java-разработчик,Java Разработчик,1.0,1,train
23,Java-разработчик,Java Разработчик,1.0,1,train
30,Java-разработчик,Ведущий инженер по разработке,1.0,1,val
31,Java-разработчик,Ведущий инженер по разработке,1.0,1,val
51,Java-разработчик,Java/Kotlin разработчик,1.0,1,train
52,Java-разработчик,Java/Kotlin разработчик,1.0,1,train
60,Senior Java-разработчик проект брокерское обсл...,Java Developer,1.0,1,val
61,Java developer,Старший Java разработчик,1.0,1,val


,title_vacancy,title_resume,label,id_cluster,split
4,Системный аналитик,Системный аналитик,1.0,2,train
5,Системный аналитик,Системный аналитик,1.0,2,train
17,Системный аналитик,Системный аналитик,1.0,2,train
18,Системный аналитик,Системный аналитик,1.0,2,train
24,Системный аналитик финтех,Ведущий системный аналитик,1.0,2,train
47,Системный аналитик,Системный аналитик,1.0,2,train
48,Системный аналитик,Системный аналитик,1.0,2,train
58,Системный аналитик,Ведущий системный аналитик,1.0,2,train
59,Системный аналитик,Ведущий системный аналитик,1.0,2,train
68,Системный аналитик,Системный аналитик,1.0,2,train


,title_vacancy,title_resume,label,id_cluster,split
9,DevOps,DevOps,1.0,3,train
253,DevOps,DevOps инженер,1.0,3,train
273,DevOps инженер,DevOps,1.0,3,train
308,DevOps,DevOps Engineer,1.0,3,train
352,DevOps инженер,DevOps инженер\Главный инженер CI/CD,1.0,3,train
478,DevOps,DevOps Engineer,1.0,3,train
507,DevOps,DevOps,1.0,3,train
554,DevOps,DevOps engineer,1.0,3,train
556,DevOps инженер,DevOps-инженер,1.0,3,train
610,DevOps,DevOps,1.0,3,train


,title_vacancy,title_resume,label,id_cluster,split
42,Архитектор/Системный аналитик DWH ДИР,Аналитик DWH,1.0,4,train
224,Аналитик DWH,Аналитик хранилищ данных,1.0,4,train
283,Аналитик DWH,Analyst (DWH),1.0,4,train
359,Ведущий/ Главный аналитик DWH,системный аналитик DWH,1.0,4,train
429,Ведущий/ Главный аналитик DWH,Ведущий инженер разработки / Аналитик DWH,1.0,4,train
443,Аналитик DWH,DWH Analyst,1.0,4,train
451,Архитектор/Системный аналитик DWH ДИР,Системный аналитик DWH,1.0,4,train
578,Ведущий/ Главный аналитик DWH,Аналитик DWH,1.0,4,train
666,"Аналитик DWH ""Hadoop""",Аналитик DWH,1.0,4,train
668,Архитектор/Системный аналитик DWH ДИР,Dwh analyst,1.0,4,train


,title_vacancy,title_resume,label,id_cluster,split
29,ETL Разработчик (ДИР),Ведущий инженер-программист,1.0,5,train
45,ETL Разработчик (ДИР),ETL-разработчик,1.0,5,train
216,ETL Разработчик (ДИР),Руководитель направления Data Engineer,1.0,5,train
301,ETL Разработчик (ДИР),Data engineer,1.0,5,train
565,ETL Разработчик (ДИР),Ведущий ETL-разработчик,1.0,5,train
621,"Ведущий разработчик ETL ""Hadoop""",Инженер по разработке,1.0,5,train
624,"Ведущий разработчик ETL ""Hadoop""",Главный инженер,1.0,5,train
651,ETL Разработчик (ДИР),Data Engineer,1.0,5,train
756,"Ведущий разработчик ETL ""Hadoop""",ETL Developer,1.0,5,train


,title_vacancy,title_resume,label,id_cluster,split
65,Product manager,"Product Lead направления ""Карьерные сервисы""",1.0,6,train
88,Product manager,IT & Product Operations Director,1.0,6,train
90,Product Manager,Product Lead of Payments,1.0,6,train
145,Product manager,Product Lead,1.0,6,train
233,Product manager,CPO,1.0,6,train
336,Product manager,Product manager | Part-time,1.0,6,train
366,Product Manager,Директор по Продуктам,1.0,6,train
510,Product manager,Product Owner,1.0,6,train
585,Product manager,Lead Product Manager,1.0,6,train
699,Product Manager,Product manager,1.0,6,train


,title_vacancy,title_resume,label,id_cluster,split
222,Тестировщик,инженер-тестировщик,1.0,7,train
739,Тестировщик,Инженер по тестированию ПО,1.0,7,train
828,Тестировщик,Инженер по тестированию,1.0,7,train


,title_vacancy,title_resume,label,id_cluster,split
161,Frontend Developer,Ведущий Frontend разработчик,1.0,8,train
309,Frontend Developer,Senior/TechLead Frontend Developer,1.0,8,train
406,Frontend Developer,Frontend-разработчик,1.0,8,train
434,Frontend Developer,Frontend Developer,1.0,8,train
642,Frontend Developer,Старший fronend-разработчик,1.0,8,train
738,Frontend Developer,Frontend-разработчик,1.0,8,train
749,Frontend Developer,Senior Fullstack Developer,1.0,8,train
763,Frontend Developer,Frontend разработчик,1.0,8,train
801,Frontend Developer,Senior Frontend Developer,1.0,8,train
861,Frontend Developer,Senior Frontend Developer,1.0,8,train


,title_vacancy,title_resume,label,id_cluster,split
291,Python Developer,Python developer,1.0,9,train
409,Python Developer,Lead developer,1.0,9,train
706,Python Developer,Python разработчик,1.0,9,train


In [49]:
for id_cluster, subset in df_titles.query('label == 0').groupby('id_cluster'):
    display(subset)
    print()

,title_vacancy,title_resume,label,id_cluster,split
13,ИТ-Лидер команды,Java lead,0.0,0,train
36,ИТ-лидер,Team Lead Java,0.0,0,train
67,ИТ-лидер,TeamLead - Java,0.0,0,train
91,ИТ-лидер,Team Lead,0.0,0,train
97,ИТ-Лидер команды,Senior Developer/Java Team Lead,0.0,0,train
147,ИТ-Лидер команды,Java Team Lead,0.0,0,train
156,ИТ-Лидер команды,Владелец продукта,0.0,0,train
183,Руководитель группы разработки,Team Lead,0.0,0,train
189,Руководитель группы разработки,Tech Lead,0.0,0,train
227,ИТ-лидер,Senior Java Software Engineer\ Team Lead,0.0,0,train


,title_vacancy,title_resume,label,id_cluster,split
1,Java-разработчик проект АБС,Java разработчик,0.0,1,train
8,Java-разработчик проект АБС,Java-разработчик,0.0,1,train
14,Java разработчик senior,Java разработчик,0.0,1,train
16,Java-разработчик проект АБС,Java Developer,0.0,1,train
25,Java developer,Java Developer,0.0,1,val
...,...,...,...,...,...
900,Senior Java-разработчик проект брокерское обсл...,Java Developer,0.0,1,val
903,Java-разработчик проект АБС,Java-разработчик,0.0,1,train
913,Java разработчик senior,Java developer,0.0,1,train
914,Java-разработчик,Java-разработчик,0.0,1,train


,title_vacancy,title_resume,label,id_cluster,split
7,Системный аналитик комманда Залоги,Ведущий Аналитик,0.0,2,train
10,Системный аналитик финтех,Аналитик,0.0,2,train
35,Системный аналитик комманда Залоги,Системный аналитик,0.0,2,train
43,Системный аналитик,Ведущий системный аналитик,0.0,2,train
44,Системный аналитик,Ведущий системный аналитик,0.0,2,train
...,...,...,...,...,...
906,Системный аналитик комманда Залоги,Системный аналитик,0.0,2,train
908,Системный аналитик,Ведущий аналитик,0.0,2,train
909,Системный аналитик,Ведущий аналитик,0.0,2,train
912,Системный аналитик комманда Залоги,Ведущий системный аналитик,0.0,2,train


,title_vacancy,title_resume,label,id_cluster,split
12,DevOps,Главный специалист (DevOps),0.0,3,train
21,DevOps,"DevOps, web developer",0.0,3,train
110,DevOps инженер,DevOps,0.0,3,train
121,DevOps,DevOps инженер,0.0,3,train
159,DevOps,Инженер L2,0.0,3,train
176,DevOps,DevOps Engineer,0.0,3,train
210,DevOps инженер,DevOps Engineer,0.0,3,train
219,DevOps инженер,DevOps инженер,0.0,3,train
277,DevOps инженер,DevOps Engineer,0.0,3,train
284,DevOps инженер,Главный DevOps инженер,0.0,3,train


,title_vacancy,title_resume,label,id_cluster,split
6,"Аналитик DWH ""Hadoop""",Системный аналитик DWH,0.0,4,train
26,"Аналитик DWH ""Hadoop""",Системный аналитик DWH,0.0,4,train
28,"Аналитик DWH ""Hadoop""",Data engineer/Аналитик DWH,0.0,4,train
50,Ведущий/ Главный аналитик DWH,Аналитик данных,0.0,4,train
53,Ведущий/ Главный аналитик DWH,Ведущий аналитик DWH,0.0,4,train
54,Ведущий/ Главный аналитик DWH,Аналитик,0.0,4,train
55,Ведущий/ Главный аналитик DWH,"Ведущий программист (SQL, DWH)/ Аналитик DWH",0.0,4,train
76,Аналитик DWH,Senior DWH Analyst,0.0,4,train
79,"Аналитик DWH ""Hadoop""",Системный аналитик,0.0,4,train
80,"Аналитик DWH ""Hadoop""",Аналитик DWH,0.0,4,train


,title_vacancy,title_resume,label,id_cluster,split
66,ETL Разработчик (ДИР),data engineer,0.0,5,train
82,ETL Разработчик (ДИР),ETL разработчик,0.0,5,train
105,"Ведущий разработчик ETL ""Hadoop""",Data Engineer,0.0,5,train
197,ETL Разработчик (ДИР),Старший разработчик ETL,0.0,5,train
198,"Ведущий разработчик ETL ""Hadoop""",ETL / Data Engineer,0.0,5,train
212,"Ведущий разработчик ETL ""Hadoop""",Code reviewer,0.0,5,train
213,"Ведущий разработчик ETL ""Hadoop""",Data engineer,0.0,5,train
316,"Ведущий разработчик ETL ""Hadoop""",Ведущий дата инженер,0.0,5,train
404,ETL Разработчик (ДИР),Data engineer,0.0,5,train
411,ETL Разработчик (ДИР),Data engineer,0.0,5,train


,title_vacancy,title_resume,label,id_cluster,split
40,Product manager,Product manager сайта,0.0,6,train
56,Product manager,Head of Product / Руководитель продуктового на...,0.0,6,train
122,Product manager,Менеджер продукта,0.0,6,train
125,Product manager,Продакт-менеджер,0.0,6,train
126,Product Manager,Product manager,0.0,6,train
199,Product manager,Product Owner,0.0,6,train
204,Product manager,Руководитель отдела разработки IT-проектов.,0.0,6,train
228,Product manager,Руководитель продуктового направления,0.0,6,train
319,Product manager,Head of HR Product,0.0,6,train
396,Product manager,Head of Product & Business Strategy,0.0,6,train


,title_vacancy,title_resume,label,id_cluster,split
100,Тестировщик,Инженер,0.0,7,train
103,Тестировщик,Middle QA,0.0,7,train
144,Тестировщик,Специалист по тестированию,0.0,7,train
300,Тестировщик,Инженер по тестированию,0.0,7,train
313,Тестировщик,ировщик ПО,0.0,7,train
343,Тестировщик,Инженер по эксплуатации,0.0,7,train
347,Тестировщик,QA Engineer,0.0,7,train
350,Тестировщик,"QA Engineer, ировщик",0.0,7,train
392,Тестировщик,QA middle,0.0,7,train
460,Тестировщик,ировщик ПО,0.0,7,train


,title_vacancy,title_resume,label,id_cluster,split
19,Frontend Developer,Senior Frontend Engineer,0.0,8,train
49,Frontend Developer,Frontend-разработчик,0.0,8,train
184,Frontend Developer,Frontend-разработчик,0.0,8,train
755,Frontend Developer,Frontend-разработчик,0.0,8,train
821,Frontend Developer,Frontend-разработчик,0.0,8,train
885,Frontend Developer,Frontend developer,0.0,8,train
902,Frontend Developer,"Frontend Developer (Javascript, Vue)",0.0,8,train


,title_vacancy,title_resume,label,id_cluster,split
151,Python Developer,Backend-разработчик,0.0,9,train
160,Python Developer,Python backend developer,0.0,9,train
346,Python Developer,Python-разработчик,0.0,9,train
361,Python Developer,Ведущий python Разработчик,0.0,9,train
495,Python Developer,Python Developer,0.0,9,train
702,Python Developer,Python Backend Developer,0.0,9,train
725,Python Developer,Python Backend разработчик,0.0,9,train
835,Python Developer,Главный инженер по разработке,0.0,9,train
857,Python Developer,Senior Python Programmer,0.0,9,train
889,Python Developer,Инженер-программист Python,0.0,9,train


# Модель определения важности слов в вакансиях и резюме 

In [50]:
clms = ['uuid', 'description', 'title', 'vacancy_fl']
df = pd.concat([
    df_vacancy.assign(vacancy_fl=True)[clms],
    df_resume.assign(vacancy_fl=False)[clms]
], sort=False, ignore_index=True)
df['id_cluster'] = df['title'].map(title2cluster)
np.random.seed(42)
df = df.sample(frac=1).reset_index(drop=True)
df['description_tokens'] = df['description'].parallel_apply(Tokenizer(norm=True))

# получаем stop_words
# если низкий максимальный коэффициент, слово не английское, не в списке ключевых слов из резюме
key_skills = df_resume.query('key_skills != ""')['key_skills'].dropna().parallel_apply(Tokenizer(norm=True))
key_skills = set(key_skills.explode())

X = df['description_tokens'].apply(lambda x: ' '.join(x))
y = df['id_cluster']
vec = TfidfVectorizer()
clf = LogisticRegression(n_jobs=16, multi_class='ovr', random_state=42, fit_intercept=False)
vec.fit(X)
clf.fit(vec.transform(X), y)
feature_names = vec.get_feature_names_out()
coef = np.max(clf.coef_, axis=0)
pattern = re.compile(r'[a-z]+')
mask = ~np.isin(feature_names, list(set(pattern.findall(' '.join(feature_names)))))
feature_names = feature_names[mask]
coef = coef[mask]
stop_words = set(feature_names[coef < 0.01]) - key_skills
stop_words = stop_words | {'желательно'}

# токенизация
df['description_tokens'] = df['description_tokens'].apply(lambda x: ' '.join(x))
df['description_tokens_2'] = df['description'].parallel_apply(Tokenizer(norm=True, stop_words=stop_words))
df['description_tokens_2'] = df['description_tokens_2'].apply(lambda x: ' '.join(x))

In [51]:
len(stop_words)

495

In [56]:
stop_words

{'qaсессия',
 'tна',
 'аpache',
 'автомобилей',
 'автономно',
 'авторотация',
 'авть',
 'адаптироваться',
 'азбука',
 'актуализация',
 'акционер',
 'альтернативный',
 'аналогично',
 'анимация',
 'аннотирование',
 'анонимизация',
 'анонс',
 'антарес',
 'антипрод',
 'аоп',
 'аппаратура',
 'аппликация',
 'арихтектурный',
 'архивный',
 'атака',
 'аутсорс',
 'бакэнд',
 'банковоска',
 'банковской',
 'барсума',
 'безблокировочный',
 'безопаасность',
 'биржей',
 'блокировочноить',
 'бокс',
 'более',
 'бот',
 'бронировать',
 'букмекер',
 'буфер',
 'бэкенд',
 'важной',
 'важный',
 'вебклиент',
 'вендорный',
 'вернуть',
 'версионность',
 'вертикальный',
 'весь',
 'видеоконференцсвязь',
 'видеообучение',
 'видеофай',
 'вклад',
 'вкладчик',
 'вкус',
 'вложенность',
 'вмс',
 'внесение',
 'внутренный',
 'внутрироссия',
 'водоснабжение',
 'вологда',
 'вологодский',
 'вопрос',
 'восток',
 'вставка',
 'встраиваемый',
 'встреча',
 'выгода',
 'выдернуть',
 'выкладываться',
 'вышеописанный',
 'вышеперечисл

In [57]:
df.head()

,uuid,description,title,vacancy_fl,id_cluster,description_tokens,description_tokens_2
0,5785c202-6744-3e1b-994a-d5bffc6aad14,"Критически важный проект, реализованный по зак...",Java Software Developer,False,1,критически важный проект реализовать заказ пра...,критически проект реализовать заказ правительс...
1,dd86d509-6381-363b-9da7-46f8e3f5e91b,Работаю в команде привлечения клиентов. Занима...,Java/Kotlin разработчик,False,1,работать команда привлечение клиент заниматься...,работать команда привлечение клиент заниматься...
2,d15abe9e-6d82-300e-9a4d-ed18dc371ea7,"Курирование, сопровождение и поддержание работ...",Ведущий аналитик DWH,False,4,курирование сопровождение поддержание работосп...,курирование сопровождение поддержание работосп...
3,7a762282-cffc-30e0-b046-44d987cedffd,Заказчик: ПАО « банк» Системный аналитик на пр...,Ведущий системный аналитик,False,2,заказчик пао банк системный аналитик проект ра...,заказчик пао банк системный аналитик проект ра...
4,34a77b82-e739-31e9-88ed-ad6ea07f1917,1) Создание витрин данных для различных направ...,Системный аналитик DWH,False,4,создание витрина данные для различный направле...,витрина данные для различный направление бизне...


In [58]:
df.isna().sum()

uuid                    0
description             0
title                   0
vacancy_fl              0
id_cluster              0
description_tokens      0
description_tokens_2    0
dtype: int64

In [59]:
X = df['description_tokens_2']
y = df['id_cluster']

vec = TfidfVectorizer()
clf = LogisticRegression(n_jobs=16, multi_class='ovr', random_state=42, fit_intercept=False)

vec.fit(X)
clf.fit(vec.transform(X), y)
feature_names = vec.get_feature_names_out()

In [60]:
eli5.show_weights(
    clf,
    vec=vec,
    top=(10,0),
    target_names=cluster2title[clf.classes_].values,
    feature_names=feature_names,
)

In [61]:
# i = np.random.randint(0,len(df)) #23931

# print(df.loc[i, 'title'])
# print()
# print(df.loc[i, 'uuid'])
# print(df.loc[i, 'vacancy_fl'])

# eli5.show_prediction(
#     clf, 
#     df.loc[i, 'description_tokens'], 
#     vec=vec, 
#     feature_names=feature_names, 
#     targets=[df.loc[i, 'id_cluster']],
#     target_names=clf.classes_
# )

In [62]:
class SorterDescription:
    def __init__(self, clf, vec, sentenizer, tokenizer):
        self._clf = clf
        self._vec = vec
        self._sentenizer = sentenizer
        self._tokenizer = tokenizer
        
    def __call__(self, description, id_cluster):
        coef = self._clf.coef_[self._clf.classes_ == id_cluster].T
        # coef[coef < 0] = 0

        sents = np.array(self._sentenizer(description))
        sents_tokenized = np.array([' '.join(self._tokenizer(text)) for text in sents])
        X = self._vec.transform(sents_tokenized)
        weight = X @ coef
        ids = np.argsort(-weight.squeeze())
        description_sorted = ' '.join(sents[ids])
        return description_sorted
    
    
class Tagger:
    def __init__(self, clf, vec, tokenizer):
        self._clf = clf
        self._vec = vec
        self._tokenizer = tokenizer
        
    def __call__(self, description, id_cluster, k=0.1):
        coef = self._clf.coef_[self._clf.classes_ == id_cluster].squeeze()
        
        sents = [' '.join(self._tokenizer(description))]                 
        X = self._vec.transform(sents).toarray().squeeze()
        mask = (X != 0) & (coef > 0)
        if mask.sum():
            weights = coef[mask] * X[mask]

            tokens = self._vec.get_feature_names_out()
            tokens = tokens[mask]

            ids = np.argsort(-weights)
            tokens = tokens[ids]
            weights = weights[ids]
            tokens = list(tokens[weights > np.quantile(weights, k)])
        else:
            tokens = []
        return tokens

sort_description = SorterDescription(clf, vec, sentenizer, Tokenizer(norm=True))
get_tags = Tagger(clf, vec, Tokenizer(norm=True))

In [63]:
df_vacancy['description_tokens'] = df_vacancy['description'] \
    .parallel_apply(Tokenizer(norm=True)) \
    .apply(lambda x: ' '.join(x))

df_resume['description_tokens'] = df_resume['description'] \
    .parallel_apply(Tokenizer(norm=True)) \
    .apply(lambda x: ' '.join(x))

df_vacancy['id_cluster'] = df_vacancy['title'].map(title2cluster)
df_resume['id_cluster'] = df_resume['title'].map(title2cluster)

df_vacancy['description_sorted'] = df_vacancy.parallel_apply(lambda x: sort_description(x.description, x.id_cluster), axis=1)
df_resume['description_sorted'] = df_resume.parallel_apply(lambda x: sort_description(x.description, x.id_cluster), axis=1)

df_vacancy['tags'] = df_vacancy.parallel_apply(lambda x: get_tags(x.description, x.id_cluster), axis=1)
df_resume['tags'] = df_resume.parallel_apply(lambda x: get_tags(x.description, x.id_cluster), axis=1)

In [64]:
i = np.random.randint(0,len(df_vacancy)) #23931

print(df_vacancy.loc[i, 'title'])
print()
print(df_vacancy.loc[i, 'uuid'])

eli5.show_prediction(
    clf, 
    df_vacancy.loc[i, 'description_tokens'], 
    vec=vec, 
    feature_names=feature_names, 
    targets=[df_vacancy.loc[i, 'id_cluster']],
    target_names=clf.classes_
)

Frontend Developer

259bf318-e6a7-3b6c-93f9-e1804a89ee63


In [65]:
i = np.random.randint(0,len(df_resume)) #23931

print(df_resume.loc[i, 'title'])
print()
print(df_resume.loc[i, 'uuid'])

eli5.show_prediction(
    clf, 
    df_resume.loc[i, 'description_tokens'], 
    vec=vec, 
    feature_names=feature_names, 
    targets=[df_resume.loc[i, 'id_cluster']],
    target_names=clf.classes_
)

Senior Software Developer Team Lead

9fe9cded-6120-384e-8f9a-96ce33bc99dd


In [66]:
i = np.random.randint(0,len(df_vacancy))

description = df_vacancy.loc[i]['description']
description_sorted = df_vacancy.loc[i]['description_sorted']
title = df_vacancy.loc[i]['title']
id_cluster = df_vacancy.loc[i]['id_cluster']

print(title)
print()
print(*sentenizer(description), sep='\n')
print()
print(*sentenizer(description_sorted), sep='\n')
print()
print(get_tags(description, id_cluster, k=0.1))

Product Manager

Февраль 2024г, профили в работе: B2B - опыт в финтехе 1)  две вакансии сюда немного разные по профилю ( ЛК - привлечение B2B партнёров и.
ВЭД ) __________________________________________________________________________ B2B - можно без финтех опыта 2) - лояльность (B2B) , есть еще одна позиция на эквайринг (интернет, не торговый) , но там пока описание не сделали еще (то есть здесь 2 позиции тоже) ____________________________________________________________________________ B2C - желателен опыт с маркетплейсами 3).
Подробнее про B2C (про нее не успели вчера).
По требованиям +- тоже самое только очень важен опыт работы с маркетплейсами.
Продукт: Кредитование внутри маркетплейса (рассрочка) - кредитования физ лиц (покупателей маркетплейса).
Задача: построить/проработать стратегию продукта (важен ретеншн и прирост новых клиентов на направление).
Направление у нас новое.
Все члены команды важные для создания продукта +- уже есть или будут в ближ время, но сейчас самое главно

In [67]:
i = np.random.randint(0,len(df_resume))

description = df_resume.loc[i]['description']
description_sorted = df_resume.loc[i]['description_sorted']
title = df_resume.loc[i]['title']
id_cluster = df_resume.loc[i]['id_cluster']

print(title)
print()
print(*sentenizer(description), sep='\n')
print()
print(*sentenizer(description_sorted), sep='\n')
print()
print(get_tags(description, id_cluster, k=0.1))

Системный аналитик WFM

Достижения: Для компании.
Глобус сделал work around файловый обмен данных между BI и WFM.
В данном интерфейсе используется библиотеки анализа данных на python для преобразования данных в формат WFM.
Данное work around решение, уменьшило время подготовки и загрузки данных в систему по 1 гипермаркету с 21 дня до 4х. Также на базе данного решения был реализован ETL процесс с ежедневной загрузкой данных в систему.
Также была разработана модель прогноза оборота в рублях и штуках, с детализацией по дням по всем магазинам компании.
При оценки произвольных периодов на уровне дня по всем магазинам взвешенная абсолютная ошибка составила 8% Обязанности: - Работа с бизнес требованиями: анализ, уточнение и согласование;
- Оценка технических альтернатив для бизнес требований;
- Составление технических заданий, совместно с разработчиками, аналитиками бизнес процессов, архитекторами и заказчиками, также участие в составлении технических спецификаций для обмена данными;
- Взаимо

In [ ]:
plot_wordcloud(df_vacancy['tags'].explode())

# Подготовка данных для финальной модели

In [26]:
clms = ['uuid', 'description_sorted', 'tags', 'title']
df = df_pairs \
    .merge(df_vacancy[clms], left_on='uuid_vacancy', right_on='uuid') \
    .drop('uuid', axis=1) \
    .merge(df_resume[clms], left_on='uuid_resume', right_on='uuid', suffixes=['_vacancy', '_resume']) \
    .drop('uuid', axis=1)

df['id_cluster'] = df['title_vacancy'].map(title2cluster)
np.random.seed(42)
df = df.sample(frac=1).reset_index(drop=True)

In [27]:
df.to_pickle(f'{DATA_PROCESSED}/df.pickle')

In [28]:
df.head()

,uuid_vacancy,uuid_resume,label,split,description_sorted_vacancy,tags_vacancy,title_vacancy,description_sorted_resume,tags_resume,title_resume,id_cluster
0,779f3a59-206a-3241-adc4-d7db504f960b,2b5ad5e1-1f31-3f3f-8a66-43cd89233672,0.0,train,Опыт разработки на Java от 3 лет. Опыт коммерч...,"[java, spring, boot, kafka, kotlin, docker, оп...",Java разработчик команда Инвестиции,"Стек ядра: Java 1.8, Spring(DI, MVC), Hibernat...","[java, spring, kotlin, boot, hibernate, mongod...",Начальник отдела java разработки,1
1,61a5a940-c9f2-3f9f-bbda-9cf735697878,843b47a5-ee84-3fec-bb03-f86b986e2a55,0.0,train,"Стрим, как. Более 10 команд в стриме. Решать а...","[стрим, руб, прекрасный, принятие, чувство, са...",ИТ-Лидер команды,Oracle Certified Associate - Java SE8 Programm...,"[programmer, arm, programming, principles, acc...",Lead Java,0
2,b2315867-73a2-3d43-acac-cbb92bd793b3,4f4ca6da-a023-3dc0-9e11-1378ff89109c,0.0,val,Gradle/Maven; - Понимание Reactive Spring (Spr...,"[spring, java, maven, kafka, приложение, rabbi...",Senior Java-разработчик проект брокерское обсл...,"Java, SQL, Java Spring Framework, Apache Kafka...","[spring, java, apache, boot, рефакторинг, hibe...",Старший Java-разработчик,1
3,dfaf7cc8-3726-361e-b7cf-6d9746d6bf77,2e38eb42-6f68-3635-899b-aa2b8f6b0620,0.0,train,Требования для QA: Понимание теории тестирован...,"[тестирование, регрессионный, фкр, тестовый, к...",Тестировщик,"SQL, PostgreSQL, Postman, Android studio, Char...","[тестирование, ирование, дефект, кейс, регресс...",QA Engineer,7
4,aecfdaf6-e12c-3309-8f1b-157028ef63d5,27427f04-a737-3679-a39c-abe5e8b13dba,0.0,train,Опыт разработки на Java с использованием техно...,"[java, spring, boot, maven, hibernate, junit, ...",Java-разработчик,"Стек технологий: Java 8, Spring Framework, Hib...","[spring, java, приложение, boot, hibernate, ka...",Senior Java Developer,1
